# AMS Benchmark

Reference:

1. Power Grid Lib - Optimal Power Flow, https://github.com/power-grid-lib/pglib-opf

In [1]:
import os
from tqdm import tqdm

import numpy as np

import andes
import ams

import pypower.api as pyp

import datetime

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'andes:{andes.__version__}')
print(f'ams:{ams.__version__}')

Last run time: 2024-03-25 23:46:05
andes:1.9.1.post24+g7a87ad5d
ams:0.9.5.post1+g114d11e


In [3]:
andes.config_logger(stream_level=50)
ams.config_logger(stream_level=50)

In [4]:
if not os.path.exists('pglib-opf'):
    !git clone https://github.com/power-grid-lib/pglib-opf.git
else:
    print('The directory "pglib-opf" already exists, skipping clone.')

The directory "pglib-opf" already exists, skipping clone.


In [5]:
directory = './pglib-opf/api/'

# List all files and directories within the specified directory
entries = os.listdir(directory)

# Get a list of files in the directory along with their sizes
files_with_sizes = [(file, os.path.getsize(os.path.join(directory, file))) for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))]

# Sort the list of files by size (the second element of the tuple)
sorted_files_with_sizes = sorted(files_with_sizes, key=lambda x: x[1])

# If you just want a list of filenames sorted by size
sorted_files = [file for file, size in sorted_files_with_sizes]

In [6]:
failed_cases = []
non_optimal_cases = []
non_identical_cases = []

n_cases = len(sorted_files)
n_cases = 20 # test partial cases
for case_i in tqdm(range(n_cases)):
    opf_case = sorted_files[case_i]
    sp = ams.load(directory + opf_case,
                  setup=True, no_output=True, default_config=True)
    # --- PYPOWER ---
    ppc = ams.io.pypower.system2ppc(sp)
    ppopt = pyp.ppoption(VERBOSE=0, OUT_ALL=0, PF_ALG=1, OPF_ALG_DC=200)
    try:
        sp.DCOPF.run(solver='MOSEK')
        ppc_sol = pyp.rundcopf(ppc, ppopt)
        aBus_check = np.allclose(a=sp.DCOPF.aBus.v * andes.shared.rad2deg,
                                b=ppc_sol['bus'][:, 8],
                                atol=1e-3)
        if not aBus_check:
            non_identical_cases.append(opf_case)
            continue
        if sp.DCOPF.exit_code != 0:
            non_optimal_cases.append(opf_case)
            continue
    except Exception as e:
        failed_cases.append(opf_case)
        continue

100%|██████████| 20/20 [00:04<00:00,  4.26it/s]


In [7]:
print(non_identical_cases)

['pglib_opf_case60_c__api.m', 'pglib_opf_case197_snem__api.m', 'pglib_opf_case500_goc__api.m']


In [8]:
print(failed_cases)

[]


In [9]:
print(non_optimal_cases)

[]
